<a href="https://colab.research.google.com/github/dportillap/htp_megathyrsus/blob/main/NetCDF4_structure_V9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias asociadas

## Instalación

In [ ]:
!sudo apt-get update
!pip install netCDF4
!pip install numpy
!pip install json
!pip install datetime
!pip install xarray
!sudo apt-get install -y netcdf-bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 14.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packa

# Definición de la estructura del archivo NetCDF4

In [ ]:
# --- // Importar los módulos necesarios

import netCDF4
import numpy as np
from datetime import datetime # para la historia de atributos
from google.colab import drive
import os

# --- // Rutas a archivos en Colab ---

# Monter el drive en colab
drive.mount('/content/drive')
# Cambio de la ruta de trabajo
os.chdir('/content/drive/My Drive/htp_megathyrsus/netcdf4_htp')
# Definir la ruta al archivo HDF5
ruta_archivo_nc = 'fenotipificacion_megathyrsus_m_nus.nc'

# --- // Información base del ensayo ---

# Cultivares y diseño experimental
nombres_bloques = [f'bloque_{i}' for i in range(1, 5)]
nombres_parcelas_por_bloque = [f'parcela_{i}' for i in range(1, 12)]

# Asignación 44 elementos en total, representados por 4 bloques y 11 parelas
asignacion = [
    # Bloque 1
    'c_326095', 'c_606866', 'c_11416058', 'c_396501', 'c_11816049', 'c_726912',
    'c_676898', 'c_356299', 'c_sabanera', 'c_12613035', 'c_646891',
    # Bloque 2
    'c_12613035', 'c_11816049', 'c_326095', 'c_646891', 'c_676898',
    'c_sabanera', 'c_11416058', 'c_606866', 'c_726912', 'c_356299', 'c_396501',
    # Bloque 3
    'c_sabanera', 'c_11416058', 'c_396501', 'c_606866', 'c_12613035',
    'c_676898', 'c_11816049', 'c_726912', 'c_326095', 'c_646891', 'c_356299',
    # Bloque 4
    'c_11816049', 'c_726912', 'c_326095', 'c_11416058', 'c_396501',
    'c_sabanera', 'c_676898', 'c_646891', 'c_606866', 'c_12613035', 'c_356299'
]

distribucion_ensayo = {}
indice_asignacion = 0
for b_nombre in nombres_bloques:
  distribucion_ensayo[b_nombre] = {}
  for p_nombre in nombres_parcelas_por_bloque:
    if indice_asignacion < len(asignacion):
        distribucion_ensayo[b_nombre][p_nombre] = {'variedad': asignacion[indice_asignacion]}
    else:
        # Este caso idealmente no debe ocurrir si la asignación es correcta
        distribucion_ensayo[b_nombre][p_nombre] = {'variedad': 'error_asignacion'}
    indice_asignacion += 1

lista_variedades = ['c_326095', 'c_606866', 'c_11416058', 'c_396501', 'c_11816049', 'c_726912', 'c_676898', 'c_356299', 'c_sabanera', 'c_12613035', 'c_646891']

dias_postcorte_eval = ['dia_21', 'dia_28', 'dia_35', 'dia_42']
num_ciclos_eval = 5

# Descripciones y unidades de las variables de evaluación de Ground truth
# Datos de campo
gt_cam_atributos = {
    'unidades': {
        'vigor': '1', 'cobertura': 'percent', 'plagas_severidad': '1',
        'enfermedad_severidad': '1', 'floracion': 'percent', 'total_peso_verde': 'kg',
        'muestra_peso_verde': 'g', 'muestra_peso_seco': 'g', 'altura': 'cm',
        'valor_spad': '1',
        'descuento_saco': '1',
        'peso_saco': 'g',
        'peso_bolsa': 'g'
    },
    'descripcion': {
        'vigor': 'Medición cualitativa de adaptación considerando cobertura, verdosidad, severidad de afectación e infestación de plagas, severidad de afectación de enfermedades y biomasa sobre el suelo',
        'cobertura': 'Porcentaje del área superficial cubierta por biomasa verde',
        'plagas_severidad': ' Medición cualitativa de la severidad en la infestación y daño causados por plagas',
        'enfermedad_severidad': 'Medición cualitativa de la severidad en la afectación por enfermedades',
        'floracion': ' Porcentaje de plantas florecidas',
        'total_peso_verde': 'Peso total de la biomasa sobre el suelo a una altura de corte de 30 cm',
        'muestra_peso_verde': 'Peso de una muestra de biomasa sobre el suelo',
        'muestra_peso_seco': 'Peso de la muestra de biomasa sobre el suelo después de secado en horno a 60° durante 72 horas',
        'descuento_saco': 'Variable binaria que describe si el peso del saco fue descontado del peso total verde',
        'peso_saco': 'Peso del saco donde el total de la biomasa verde fue pesada',
        'peso_bolsa': 'Peso de la bolsa donde se la muestra de biomasa verde fue pesada',
        'altura': 'Altura de la hoja bandera en 4 plantas',
        'valor_spad': 'Valor de medición SPAD en 3 hojas.'
    }
}
gt_lab_atributos = {
    'unidades': {
        'proteina_cruda': 'percent', 'fdn': 'percent', 'fda': 'percent', 'digestibilidad_ms': 'percent'
    },
    'descripcion': {
        'proteina_cruda': 'Proteina cruda como porcentaje de la materia seca estimado con química húmeda',
        'fdn': 'Fibra en detergente neutrp como porcentaje de la materia seca estimado com química húmeda',
        'fda': 'Fibra en detergente ácido como porcentaje de la materia seca estimado con química húmeda',
        'digestibilidad_ms': 'Digestibilidad de la materia seca expresado como porcentaje y estimado con química húmeda in vitro'
    }
}

# -- // Geotransformación para los rásters

# Redefinir según se considere el ráster de referencia. Para lograr la interoperabilidad entre aplicaciones, se deja un atributo de geotransformación en cada ráster y el WKT del sistema de referencia para la correcta lectura por GDAL. La referencia utilizando el estándar CF con la ubicación del sistema de referencia en Root referido por el atributo grid_mapping sirve para programas de análisis científico, mientras que la geotransformación es útil para programas GIS más orientados a la visualización.

gt = {'X_coord_superior_izquierdo': 517476.01061331073,
      'X_resolucion_metros': 0.01,
      'Y_coord_superior_izquierdo': 716481.1415001423,
      'Y_resolucion_metros': -0.01,
      'X_numero_pixeles_width': 9098,
      'Y_numero_pixeles_height': 8159
      }

ulx = gt['X_coord_superior_izquierdo']
x_res = gt['X_resolucion_metros']
uly = gt['Y_coord_superior_izquierdo']
y_res = gt['Y_resolucion_metros']
width = gt['X_numero_pixeles_width']
height = gt['Y_numero_pixeles_height']

geotransform_tuple = (ulx, x_res, 0.0, uly, 0.0, y_res)
geotransform_string = " ".join(map(str, geotransform_tuple))

# WKT del sistema de referencia
wkt_string = 'PROJCS["WGS_84_UTM_Zone_18N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["false_easting",500000.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",-75.0],PARAMETER["scale_factor",0.9996],PARAMETER["latitude_of_origin",0.0],UNIT["Meter",1.0]]'

# Cálculo de las coordenadas para los centros de los pixeles
# Coordenadas X
# Para los centros de píxel, adicionar la mitad del acho de l pixel a la coordenada del borde
x_coords_centers = ulx + (np.arange(width) * x_res) + (x_res / 2)
# Coordenadas Y
# Para los centros de píxel, adicionar la mitad del alto de l pixel a la coordenada del borde
y_coords_centers = uly + (np.arange(height) * y_res) + (y_res / 2)

# ---// Creación del archivo NetCDF5

# Utilizar el modo 'w' reescribirá el archivo si existe. Usar 'a' para addicionar

with netCDF4.Dataset(ruta_archivo_nc, 'w', format='NETCDF4') as ncfile:

  # --- // ROOT ---
  # Atributos globales

  ncfile.title = 'Datos de fenotipificación de alto rendimiento (HTP) en campo para cultivares de Megathyrsus maximus bajo condiciones de ladera en el nordeste antioqueño'
  ncfile.institution = 'Agrosavia Centro de Investigación El Nus'
  ncfile.source = 'Datos obtenidos por sensores a bordo de vehículos aéreos no tripulados de grado consumidor, receptores GNSS de precisión, medidas de la verdad terrestre en campo y resultados de laboratorio de muestras de biomasa'
  ncfile.history = f'Creado en: {datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")}'
  # Conveniones Climate and Forecast guias para archvos NetCDF
  ncfile.Conventions = 'CF-1.8'
  ncfile.version_file_format = '2.0_NetCDF'
  ncfile.general_description = 'Archivo NetCDF4 para los datos de fenotipificación en campo de variedades de Megathyrsus maximus obtenidos en el Centro de Investigación El Nus de Agrosavia'
  ncfile.comment = 'La estructura del archivo se encuentra orientada para analisis multitemporales comparativos. Bloques parccelas areas de siembra y ROI se almacenan como subconjuntos de rásters que representan ortofotomosaicos obtenidos durante la duración del ensayo'

  # --- Definir las dimensiones comunes en el nivel Root ---

  # Estas dimensiones son utilizadas por las variables en el archivo
  ncfile.createDimension('num_bloque', len(nombres_bloques))
  ncfile.createDimension('num_parcela', len(nombres_parcelas_por_bloque))
  ncfile.createDimension('num_dias_postcorte', len(dias_postcorte_eval))
  ncfile.createDimension('num_ciclos', num_ciclos_eval)
  ncfile.createDimension('num_variedades', len(lista_variedades))
  ncfile.createDimension('num_coord_xy', 2)
  ncfile.createDimension('num_ms_bandas', 4)
  ncfile.createDimension('num_rgb_bandas', 3)
  ncfile.createDimension('num_slices', 4)
  ncfile.createDimension('num_medidas_repetidas', None)
  ncfile.createDimension('num_puntos', None)
  ncfile.createDimension('num_spectral_indices',11)
  ncfile.createDimension('num_glcm_estadisticos', 9)

   # CRS del ensayo
  crs_utm18n_var = ncfile.createVariable('crs_utm18n', 'i4')
  crs_utm18n_var.grid_mapping_name = 'transverse_mercator'
  crs_utm18n_var.longitude_of_central_meridian = -75.0
  crs_utm18n_var.latitude_of_projection_origin = 0.0
  crs_utm18n_var.false_easting = 500000.0
  crs_utm18n_var.false_northing = 0.0
  crs_utm18n_var.scale_factor_at_central_meridian = 0.9996
  crs_utm18n_var.semi_major_axis = 6378137.0
  crs_utm18n_var.inverse_flattening = 298.257223563
  crs_utm18n_var.projected_crs_name = 'WGS 84 / UTM zone 18N'
  crs_utm18n_var.epsg_code = 'EPSG:32618'
  # WKT string
  crs_utm18n_var.spatial_ref = wkt_string

  # Definir las dimensiones para los raster para la exensión completa del área experimental

  # Dimension Y para el ráster
  ncfile.createDimension('y_r', height)
  # Dimesion X para el raster
  ncfile.createDimension('x_r', width)

  # Creación de variables coordenada

  # Localizaciones espaciales de las dimensiones X y Y para los rasters
  y_r_coord_var = ncfile.createVariable('y_r', 'f8', ('y_r',))
  y_r_coord_var.long_name = 'coordenada Y en UTM zona 18N'
  y_r_coord_var.standard_name = 'projection_y_coordinate'
  y_r_coord_var.units = 'm'
  y_r_coord_var.axis = 'Y'

  x_r_coord_var = ncfile.createVariable('x_r', 'f8', ('x_r',))
  x_r_coord_var.long_name = 'coordenada X en UTM zona 18N'
  x_r_coord_var.standard_name = 'projection_x_coordinate'
  x_r_coord_var.units = 'm'
  x_r_coord_var.axis = 'X'

  # Valores para las variables coordenada que definen la extensión y ubicación de los pixeles en las imágenes

  print("Shape of x_coords_centers:", x_coords_centers.shape)
  print("Sample of x_coords_centers:", x_coords_centers[:5])
  print("Shape of y_coords_centers:", y_coords_centers.shape)
  print("Sample of y_coords_centers:", y_coords_centers[:5])
  print("Attempting to write to x_r_coord_var and y_r_coord_var...")
  x_r_coord_var[:] = x_coords_centers
  y_r_coord_var[:] = y_coords_centers
  print("Successfully wrote to x_r and y_r coordinate variables.")

  # Localizaciones espaciales de las dimensiones X y Y para vectores
  v_coord_var = ncfile.createVariable('num_coord_xy', str , ('num_coord_xy'))
  v_coord_var.long_name = 'Coordenadas X, Y en UTM zona 18N para representación de vectores'
  v_coord_var[:] = np.array(['X', 'Y'], dtype=object)

  # Dimensiones ilimitadas para las series de tiempo
  ncfile.createDimension('num_tiempo_hora', None)
  ncfile.createDimension('num_tiempo_diario', None)

  # Variable coordenada del ciclo
  ciclo_id_var = ncfile.createVariable('num_ciclos', str, ('num_ciclos',))
  ciclo_id_var.long_name = 'Identificador del ciclo de evaluación'
  # Actualizar con nombres de ciclo
  ciclo_id_var[:] = np.array([f'ciclo_{i}' for i in range(1, num_ciclos_eval + 1)], dtype=object)

  # Días post corte de la evaluación
  dias_pc_eval_coor = ncfile.createVariable('num_dias_postcorte', str, ('num_dias_postcorte',))
  dias_pc_eval_coor[:] = np.array(dias_postcorte_eval, dtype=object)
  dias_pc_eval_coor.long_name = 'Dias despues del corte de uniformidad cuando las evaluaciones fueron realizadas'

  # Descripción de las variedades en el estudio
  variedades_coor = ncfile.createVariable('num_variedades', str, ('num_variedades',))
  variedades_coor[:] = np.array(lista_variedades, dtype=object)
  variedades_coor.long_name = 'Lista de cultivares identificados en el ensayo'
  variedades_coor.cf_role = 'timeseries_id'

  # Creación de variables coordenadas para mantener los identificadores
  # Bloques
  bloques_coord = ncfile.createVariable('num_bloque', str, ('num_bloque',))
  bloques_coord[:] = np.array(nombres_bloques, dtype=object)
  # Parcelas
  parcelas_coord = ncfile.createVariable('num_parcela', str,
  ('num_parcela',))
  parcelas_coord[:] = np.array(nombres_parcelas_por_bloque, dtype=object)

  # Variable coordenada para los indices de corte
  slice_coord = ncfile.createVariable('num_slices', str, ('num_slices',))
  slices = ['y_inicial', 'y_final', 'x_inicial', 'x_final']
  slice_coord[:] = np.array(slices, dtype=object)
  slice_coord.long_name = 'indice a las coordenadas del raster para extraer pixeles de parcelas y rois'

  # Variable coordenada para puntos
  puntos_coord = ncfile.createVariable('num_puntos', 'u4', ('num_puntos',))
  puntos_coord.long_name = 'Indicadores a los puntos en vectores en orden si es necesario para describir polígonos'

  # Variable coordenada para medidas repetidas como altura y spad
  medidas_repetidas_coord = ncfile.createVariable('num_medidas_repetidas', 'u4', ('num_medidas_repetidas',))
  medidas_repetidas_coord.long_name = 'Indicadores a las medidas repetidas'

  # Variable coordenada para el espectro para bandas multiespectrales
  bandas_ms_coord = ncfile.createVariable('num_ms_bandas', str, ('num_ms_bandas',))
  bandas_ms_coord[:] = np.array(['green', 'red', 'red_edge', 'nir'], dtype=object)
  bandas_ms_coord.long_name = 'Nombre de bandas multiespectrales asociadas al sensor'
  bandas_ms_coord.comment = 'Longitud de onda central(nm)  green: 560 red: 650 red_edge:730 nir:860'

  # Variable coordenada para las bandas RGB
  bandas_rgb_coord= ncfile.createVariable('num_rgb_bandas', str, ('num_rgb_bandas',))
  bandas_rgb_coord[:] = np.array(['red', 'green', 'blue'], dtype=object)
  bandas_rgb_coord.long_name = 'Nombre de bandas RGB asociadas al sensor'

  # Variable coordenada para indices espectrales generados
  indices_espectrales_coord = ncfile.createVariable('num_spectral_indices', str, ('num_spectral_indices',))
  indices_espectrales_coord.long_name = 'Acrónimos de índices espectrales relevantes generados por las bandas green, red, red edge y nir '
  indices_espectrales_coord[:] = np.array(['ndvi', 'gndvi', 'ndre', 'ci_rededge', 'ci_green', 'rvi', 'sr_rededge', 'evi2', 'savi', 'msavi', 'grvi'])
  indices_espectrales_coord.comment = 'ndvi: Índice de Vegetación de Diferencia Normalizada ((NIR - R) / (NIR + R)); gndvi: Índice Verde de Vegetación de Diferencia Normalizada ((NIR - G) / (NIR + G)); ndre: Índice de Diferencia Normalizada del Borde Rojo ((NIR - RE) / (NIR + RE)); ci_rededge: Índice de Clorofila del Borde Rojo ((NIR / RE) - 1); ci_green: Índice de Clorofila Verde ((NIR / G) - 1); rvi: Índice de Vegetación de Ratio (NIR / R); sr_rededge: Índice de Ratio Simple del Borde Rojo (NIR / RE); evi2: Índice de Vegetación Mejorado de Dos Bandas (2.5 * (NIR - R) / (NIR + 2.4*R + 1)); savi: Índice de Vegetación Ajustado al Suelo (((NIR - R) / (NIR + R + L)) * (1 + L), con L=0.5 comúnmente); msavi: Índice de Vegetación Ajustado al Suelo Modificado ((2*NIR + 1 - sqrt((2*NIR + 1)**2 - 8*(NIR - R))) / 2); grvi: Índice de Vegetación Verde-Rojo ((G - R) / (G + R))'

  # Variables coordenada para estadísticas de textura GLCM Matriz de co-ocurrencia de niveles de gris
  glcm_estadisticos_coord = ncfile.createVariable('num_glcm_estadisticos', str, ('num_glcm_estadisticos',))
  glcm_estadisticos_coord.long_name = 'Acrónimos de estadísticas de textura GLCM Matriz de co-ocurrencia de niveles de gris'
  glcm_estadisticos_coord.comment = 'GLCM o Gray-Level Co-ocurrence Matrix. Método estadístico para describir la textura de una imagen. La textura se refiere a la organización espacial de las intensidades del pixel (niveles de gris). Patrones como suave, tosco, regular o aleatorio. Las estadísticas se derivan de una tabla 2D que permite entender con qué frecuencia los valores de diferentes niveles de gris se encuentran en relaciones espaciales específicas uno con otro. Los estadísticos que se consideran en la variable son: cont: Contraste (Contrast); diss: Disimilitud (Dissimilarity); homo: Homogeneidad (Homogeneity / Inverse Difference Moment, IDM); asm: Momento Angular Segundo (Angular Second Moment, ASM); ener: Energía (Energy - a menudo calculada como la raíz cuadrada de asm); corr: Correlación (Correlation); entr: Entropía (Entropy): glcm_media: media de la GLCM; glcm: varianza de la GLCM'
  glcm_estadisticos_coord[:] = np.array(['cont', 'diss', 'homo', 'asm', 'ener', 'corr', 'entr', 'glcm_media', 'glcm_varianza'], dtype=object)

  # Variables coordenada para datos temporales

  # Tiempo en horas
  time_hourly = ncfile.createVariable('num_tiempo_hora', 'f8', ('num_tiempo_hora'))
  time_hourly.long_name = 'Medida de tiempo en horas desde la fecha de siembra'
  time_hourly.standard_name = 'time'
  time_hourly.units = 'hours since 2024-12-31 00:00:00'
  time_hourly.calendar = 'gregorian'
  time_hourly.axis = 'T'
  time_hourly.comment = 'Horas después de la fecha de siembra 2024-12-31 00:00:00 '

  # Tiempo en días
  time_daily = ncfile.createVariable('num_tiempo_diario', 'f8', ('num_tiempo_diario'))
  time_daily.long_name = 'Medida de tiempo en días después de la fecha de siembra'
  time_daily.standard_name = 'time'
  time_daily.units = 'days since 2024-12-31 00:00:00'
  time_daily.calendar = 'gregorian'
  time_daily.axis = 'T'
  time_daily.comment = 'Dias después de la fecha de siembra 2024-12-31 00:00:00 '
  # --- // GRUPO METADATOS ----

  # Grupo metadatos
  meta_grp = ncfile.createGroup('metadatos_ensayo')
  meta_grp.nombre_del_ensayo = 'Ensayo para la fenotipificación de cultivares de Megathyrsus maximus bajo condiciones de ladera en el nordeste antioqueño'
  meta_grp.localizacion_general = 'Lote Altamira, C.I. El Nus'
  meta_grp.fecha_siembra = '2024-10-31'
  meta_grp.descripcion_ensayo = 'Selección de cultivares promisorios de M. maximus en el C.I. El Nus a partir de la fenotipificación en campo de parcelas distribuidas en un diseño de bloques completos al azar'
  meta_grp.crs_general_experimental_setup = 'EPSG:32618'
  meta_grp.objetivos_experimento = 'Evaluar la adaptación y productividad de 11 cultivares de la gramínea forrajera Megatyrsus maximus bajo condiciones de ladera en la región del nordeste antioqueño utilizando enfoques de fenotipificación de alto rendimimiento (High-Throughput Phenotyping HTP) con el uso de vehículos aéreos no tripulados (UAV) y receptores GNSS de precisión'
  meta_grp.diseno_experimental = 'Diseño de bloques completos al azar con 4 repeticiones. El factor de bloqueo se definió respecto a la ubicación en la pendiente'
  meta_grp.detalles_siembra = 'Siembra con patrón triangular (tresbolillo) con una distancia entre plantas aproximada de 0.3m'
  meta_grp.metodo_muestreo = 'Se consideraron ciclos de evaluación como las réplicas de una secuencia de mediciones hechas después de 21, 28, 35 y 42 días después de un corte de uniformidad realizado en el día 0. Las mediciones en cada día posterior al corte se realizaron en un sitio distinto de la parcela identificado en un ROI ubicado en uno de los cuatro cuadrantes de la parcela buscando dejar por fuera las plantas del borde. Los ROI fueron delimitados por marco cuadrado con un área de 1 metro cuadrado y ubicados suelo representando áreas superficiales. La ubicación de los ROI se definió utilizando receptores GNSS de posicionamiento preciso referidas a un punto de referencia con una configuración RTK Base-Rover con comunicación UHF y lineas base de máximo 60 metros. En cada ROI se realizó una evaluación cualitativa de vigor, cobertura, afectación de plagas y enfermedades, altura de cuatro plantas a su hoja bandera, lecturas del sensor SPAD en tres plantas al azar. Igualmente se cortó dentro en cada ROI la biomasa de las plantas a 30 cm de altura desde el suelo independientemente de si se ubicara parte de la biomasa superior fuera del marco de muestreo y de la entrada de la biomasa superior de otras plantas hacia dentro del marco, la cuale no fue considerada. Esta condición está definida en la metodología convencional de evaluación de forrajes en la que se enmarcó el ensayo y puede limitar análisis de asociacion con rasters obtenidos con UAV. Del corte de cada ROI se obtuvo el peso total fresc de biomasa, el peso fresco de una submuestra y el peso seco de la misma después de llevarla al horno por 72 horas a 60°C. Parte de las muestras se llevaron a laboratorio para análisis bromatológico'

  # Ubicación del centroide del sitio
  coords_sitio = meta_grp.createVariable('coordenadas_centroide_sitio', 'f8', ('num_coord_xy',))
  coords_sitio[:] = np.array([517521.3426974837, 716441.0576309565])
  coords_sitio.long_name = 'Coordenadas planas (x, y) del centroide del área experimental'
  coords_sitio.units = 'metros_este metros_norte'
  coords_sitio.grid_mapping = 'crs_utm18n'

  # Elevación del sitio (escalar: sin dimensiones asociadas)
  elevacion_sitio = meta_grp.createVariable('elevacion_sitio', 'f8')
  elevacion_sitio[:] = 0.0
  elevacion_sitio.units = 'metros'
  elevacion_sitio.long_name = 'Altura ortométrica del sitio'
  elevacion_sitio.standard_name = 'height_above_mean_sea_level'
  elevacion_sitio.datum = 'GEOCOL2004'
  elevacion_sitio.ondulacion_geoidal = 17.3
  elevacion_sitio.comment = 'Elevación sobre el nivel del mar del área experimental'

  # --- // GRUPO DISEÑO EXPERIMENTAL ---

  dis_exp_grp = ncfile.createGroup('diseno_experimental')
  dis_exp_grp.descripcion = 'Datos que definen el planteamiento del ensayo y las condiciones iniciales del área experimental. Incluye rasters base, el diseño y aplicación del plan de siembra'
  dis_exp_grp.comment = 'Considera los rasters base con los que se planificaron las áreas de siembra y de referencia para generar modelos de altura de dosel. También incluye ortomosaicos realizados en la evaluación de sobrevivencia inicial, así como las parcelas definidas y las áreas sembradas en campo'
  dis_exp_grp.crs_spatial_references = 'crs_utm18n'
  dis_exp_grp.long_name = 'Configuración del experimento de fenotipificación'

  # Información relacionadas con el diseño
  fecha_corte_uniformidad = dis_exp_grp.createVariable('fecha_corte_uniformidad_ciclo', str, ('num_ciclos',))
  fecha_corte_uniformidad.long_name = 'Fecha de corte de uniformidad para cada ciclo de evaluación'
  fecha_corte_uniformidad.comment = 'Fecha del corte de uniformidad realizado en las parcelas al inicio de cada ciclo de evaluación. Strings de marca de tiempo en formato YYYY-MM-DD HH:MM:SS para cada ciclo y paso de tiempo'
  fecha_corte_uniformidad.coordinates = 'num_ciclos'

  # Función para crear variables ráster utilizando dimensiones globales y CRS
  def create_raster_variable_cf_global_dims(group, var_name, standard_name,  long_name, units, grid_mapping_var_path, fill_value, dtype='f4', zlib=True, chunksizes=(1024,1024), bands_dim=None, wkt_string=wkt_string, geotransform_string=geotransform_string):

    """ Crea una variable raster en el grupo NetCDF utilizando dimensiones globales y convenciones CF.

  Args:
      group (netCDF4.Group): El grupo NetCDF donde la variable será creada.
      var_name (str): El nombre de la variable.
      standard_name (str): El nombre estándar para la variable.
      long_name (str): Un nombre largo descriptivo para la variable.
      units (str): Las unidades de la variable.
      grid_mapping_var_path (str): La ruta al CRS grid mapping variable en el archivo NetCDF.
      fill_value (float or int): El valor a utilizad para sin datos.
      dtype (str, optional): El tipo de datos de la variable. Por defecto 'f4'.
      zlib (bool, optional): Si permite compresion zlib. Por defecto True.
      chunksizes (tuple, optional): El chunksize de la variable. Por defecto a (2048, 2048).
      bands_dim (str, optional):El nomre de la dimension de la banda. Por defecto None.

  Returns:
      Variable netCDF4: El objeto variable NetCDF creado.

  Esta función crea una variable NetCDF con dimensiones (x, y) (y opcionalmente una dimension de bandas) y define varios atributos de acuerdo a las convenciones CF, incluyendo standard_name, long_name, units_ grid_mapping, coordinates, and un lugar lara almacenar la geotransformación y las dimensiones del raster.
  """
    # Usa las dimensiones globales 'y_r' y 'x_r'
    dims = ('y_r', 'x_r')
    current_chunksizes = list(chunksizes)

    if bands_dim:
        # Adiciona la dimension de la banda si está presente
        dims = (*dims, bands_dim)
        # Adiciona un chunksize para la dimension de las bandas. Utiliza 1
        current_chunksizes.append(1)

    var = group.createVariable(var_name, dtype, dims, fill_value=fill_value, zlib=zlib, chunksizes=tuple(current_chunksizes))
    if standard_name is not None:
      var.standard_name = standard_name
    var.long_name = long_name
    if units is not None:
      var.units = units

    # Atributos de sistema de referencia en WKT y geotransformación específicos para GDAL y GIS
    var.crs_wkt = wkt_string
    var.GeoTransform = geotransform_string

    # Atributos de sistema de referencia específicos de CF
    # Ruta a la variable global CRS
    var.grid_mapping = grid_mapping_var_path
    # Asociacion a las coordenadas globales
    var.coordinates = 'y_r x_r'

    return var

  # Rásters estáticos para el área experimental utilizando x y y globales
  # Static rasters for the experimental area using global x and y

  # DEM
  dem_ae_dset = create_raster_variable_cf_global_dims(dis_exp_grp, 'modelo_terreno', 'height_above_reference_ellipsoid', 'Modelo de elevación digital para el área experimental', 'm', 'crs_utm18n', -9999.0, wkt_string=wkt_string, geotransform_string=geotransform_string)

  # Pendiente
  slope_ae_dset = create_raster_variable_cf_global_dims(dis_exp_grp, 'pendiente', 'surface_slope_angle', 'Pendiente del área experimental', 'degrees', 'crs_utm18n', -9999.0, wkt_string=wkt_string, geotransform_string=geotransform_string)

  # Aspecto
  aspect_ae_dset = create_raster_variable_cf_global_dims(dis_exp_grp, 'orientacion_pendiente', 'surface_azimuth_angle', 'Aspecto del área experimental', 'degrees', 'crs_utm18n', -9999.0, wkt_string=wkt_string, geotransform_string=geotransform_string)

  # Área superficial
  surface_ae_dset = create_raster_variable_cf_global_dims(dis_exp_grp, 'area_superficial', None, 'Área superficial por pixel', 'm2', 'crs_utm18n', -9999.0, wkt_string=wkt_string, geotransform_string=geotransform_string)

    # Índice de humedad topográfica
  wti_ae_dset = create_raster_variable_cf_global_dims(dis_exp_grp, 'twi', None, 'Índice de humedad topográfica del área experimental', '1' , 'crs_utm18n', -9999.0, wkt_string=wkt_string, geotransform_string=geotransform_string)

  # Densidad inicial de siembra
  ds_ae_dset = create_raster_variable_cf_global_dims(dis_exp_grp, 'densidad_siembra', None, 'Densidad de siembra inicial del área experimental establecida por interpolación de los sitios de siembra con la estimación density kernel a 60 cm', 'm-2', 'crs_utm18n', -9999.0, wkt_string=wkt_string, geotransform_string=geotransform_string)

  # Ortomosaico RBG del área previa a la siembra
  om_rgb_ae_pre_dset = create_raster_variable_cf_global_dims(dis_exp_grp, 'ortom_rgb_presiembra', None, 'Ortomosaico RGB del área experimental previa a la siembra', '1', 'crs_utm18n', 0, dtype='u1', bands_dim='num_rgb_bandas', wkt_string=wkt_string, geotransform_string=geotransform_string)

  # Ortomosaico MS del área previa a la siembra
  om_ms_ae_pre_dset = create_raster_variable_cf_global_dims(dis_exp_grp, 'ortom_ms_presiembra', None, 'Ortomosaico MS del área experimental previo a la siembra', '1','crs_utm18n', -9999.0, dtype='f4', bands_dim='num_ms_bandas', wkt_string=wkt_string, geotransform_string=geotransform_string)

  # Ortomosaico RGB en la evaluación de sobrevivencia
  om_rgb_ae_sob_dset = create_raster_variable_cf_global_dims(dis_exp_grp, 'ortom_rgb_sobrevivencia', None, 'Ortomosaico RGB del área experimental posterior a la siembra en la evaluación de sobrevivencia', '1', 'crs_utm18n', 0,  dtype='u1', bands_dim='num_rgb_bandas')

  # Ortomosaico multiespectral en la evaluación de la sobrevivencia
  om_ms_ae_sob_dset = create_raster_variable_cf_global_dims(dis_exp_grp, 'ortom_ms_sobrevivencia', None, 'Ortomosaico MS del área experimental posterior a la siembra en la evaluación de sobreviviencia', '1', 'crs_utm18n', -9999.0, dtype='f4', bands_dim='num_ms_bandas', wkt_string=wkt_string, geotransform_string=geotransform_string)

  # Máscaras de las areas de siembra de las parcelas
  ma_pa_ae_dset = create_raster_variable_cf_global_dims(dis_exp_grp, 'mascara_parcelas', None, 'Mascara de las parcelas que descibe pixeles con y sin datos en toda el área experimental', '1', 'crs_utm18n', 255,  dtype='u1', wkt_string=wkt_string, geotransform_string=geotransform_string)

  # Almacenar la asignación de la variedad por parcela
  # Crear variable de asignación
  variedad_parcela_var = dis_exp_grp.createVariable('variedad_asignada_parcela',str, ('num_bloque', 'num_parcela'))
  variedad_parcela_var.long_name = 'Variedad asignada a cada parcela'
  # Indica otras variales que definen estas partes de la coordenada
  variedad_parcela_var.coordinates = 'num_bloque num_parcela'

  # Asignar los valores a la variable
  # Iterar a través de los bloques
  for b_idx, nombre_bloque in enumerate(nombres_bloques):
    # Actualizar variedad_parcela_var basado en la distribución del ensayo
    # Iterar a través de cada parcela para asignar los nombres de los cultivares
    for p_idx, nombre_parcela in enumerate(nombres_parcelas_por_bloque):
      if nombre_bloque in distribucion_ensayo and nombre_parcela in distribucion_ensayo[nombre_bloque]:
        variedad_parcela_var[b_idx, p_idx] = distribucion_ensayo[nombre_bloque][nombre_parcela]['variedad']
      else:
        variedad_parcela_var[b_idx, p_idx] = 'error_asignacion'

  # Variable para alojar las coordenadas de los Bloques
  bloques_vect_coor = dis_exp_grp.createVariable('bloque_coordenadas_vertices', 'f8', ( 'num_bloque', 'num_coord_xy', 'num_puntos'), fill_value=np.nan)
  bloques_vect_coor.long_name = 'Coordenadas X Y de los bloques'
  bloques_vect_coor.units = 'm'
  bloques_vect_coor.coordinates = 'num_bloque num_coord_xy num_puntos'
  bloques_vect_coor.grid_mapping = 'crs_utm18n'
  bloques_vect_coor.featureType = 'polygon'

  # Variable para alojar las coordenadas de la siembra de las Parcelas
  parcela_siem_vect_coor = dis_exp_grp.createVariable('siembra_parcela_coordenadas_vertices', 'f8', ( 'num_bloque', 'num_parcela', 'num_coord_xy', 'num_puntos'), fill_value=np.nan)
  parcela_siem_vect_coor.long_name = 'Coordenadas X Y de las parcelas definidos por el área de influencia de la siembra'
  parcela_siem_vect_coor.units = 'm'
  parcela_siem_vect_coor.coordinates = 'num_bloque num_parcela num_coord_xy num_puntos'
  parcela_siem_vect_coor.grid_mapping = 'crs_utm18n'
  parcela_siem_vect_coor.ancillary_variables = 'variedad_asignada_parcela'
  parcela_siem_vect_coor.featureType = 'polygon'

  # Variable para alojar las coordenadas de las Parcelas planeadas
  parcela_plan_vect_coor = dis_exp_grp.createVariable('parcela_coordenadas_vertices', 'f8', ( 'num_bloque', 'num_parcela', 'num_coord_xy', 'num_puntos'), fill_value=np.nan)
  parcela_plan_vect_coor.long_name = 'Coordenadas X Y de las parcelas planeadas para la siembra'
  parcela_plan_vect_coor.units = 'm'
  parcela_plan_vect_coor.coordinates = 'num_bloque num_parcela num_coord_xy num_puntos'
  parcela_plan_vect_coor.grid_mapping = 'crs_utm18n'
  parcela_plan_vect_coor.ancillary_variables = 'variedad_asignada_parcela'
  parcela_plan_vect_coor.featureType = 'polygon'

  # Variable para alojar las coordenadas de los sitios de siembra en las parcelas
  sitios_siembra_vect_coor = dis_exp_grp.createVariable('sitios_siembra_coordenadas_puntos', 'f8', ( 'num_bloque', 'num_parcela', 'num_coord_xy', 'num_puntos'), fill_value=np.nan)
  sitios_siembra_vect_coor.long_name = 'Coordenadas X Y de las de los sitios de siembra en cada parcela dentro de cada bloque'
  sitios_siembra_vect_coor.units = 'm'
  sitios_siembra_vect_coor.coordinates = 'num_bloque num_parcela num_coord_xy num_puntos'
  sitios_siembra_vect_coor.grid_mapping = 'crs_utm18n'
  sitios_siembra_vect_coor.ancillary_variables = 'variedad_asignada_parcela'
  sitios_siembra_vect_coor.featureType = 'point'

  # Variable para alojar los indices que identifican cada bloque en la referencia de los rasters
  bloque_rast_index = dis_exp_grp.createVariable('indice_pixeles_bloque', 'i4', ('num_bloque', 'num_slices'), fill_value=-9999)
  bloque_rast_index.long_name = 'Indice para cortar las dimensiones globales a la extensión del área de los bloques evaluados'
  bloque_rast_index.comment = 'El índice es inclusivo para el inicio y exclusivo para el final'
  bloque_rast_index.coordinates = 'num_bloque num_slices'

  # Variable para alojar los indices que identifican cada parcela en la referencia de los rasters
  parcela_rast_index = dis_exp_grp.createVariable('indice_pixeles_siembra_parcela', 'i4', ('num_bloque', 'num_parcela', 'num_slices'), fill_value=-9999)
  parcela_rast_index.long_name = 'Indice para cortar las dimensiones globales a la extensión del área definida como área de influencia de la parcela'
  parcela_rast_index.comment = "Indices (inicio inclusivo, final exclusivo) para cortar las dimensiones globales 'y' y 'x' para la extension del área de influencia de la parcela dentro de los rasters del área experimental o rasters de evaluacion. El área de influencia de la parcela se estableció a partir del patrón de siembra real y la distribución de las plantas con una envolvente convexa y un buffer de 60 m para contar el desborde de las plantas en los bordes"
  parcela_rast_index.coordinates = 'num_bloque num_parcela num_slices'
  parcela_rast_index.ancillary_variables = 'variedad_asignada_parcela'

  # Rasters dinámicos correspondientes a las imagenes multiespectrales, rgb y modelos de dosel en cada dia de evaluación dentro de cada ciclo

  # --- GRUPO DATOS EVALUACIONES ---

  evaluacion_grp = ncfile.createGroup('evaluaciones')
  evaluacion_grp.title = 'Datos dinámicos de las evaluaciones'
  evaluacion_grp.description = 'Información dimámica relacionada con la evaluación de los cultivares considerando medidas en campo, rasters de los vuelos realizados, medidas en campo y resultados de laboratorio'
  evaluacion_grp.crs_spatial_references = 'crs_utm18n'

  # Variables auxiliares que definen los datos tomados en campo
  id_evaluacion = evaluacion_grp.createVariable('id_evaluacion', 'i4', ('num_ciclos', 'num_dias_postcorte'), fill_value=-9999)
  id_evaluacion.long_name = 'Identificación individual de cada evaluación en el ensayo'
  id_evaluacion.comment = 'Identificación individual de cada evaluación en el ensayo'
  id_evaluacion.coordinates = 'num_ciclos num_dias_postcorte'

  id_asignacion = evaluacion_grp.createVariable('id_asignacion', 'i4', ('num_bloque', 'num_parcela'), fill_value=-9999)
  id_asignacion.long_name = 'Identificación individual de las asignaciones bloque parcela en el ensayo'
  id_asignacion.comment = 'Identificación individual de las asignaciones bloque parcela en el ensayo'
  id_asignacion.coordinates = 'num_bloque num_parcela'
  id_asignacion.ancillary_variables = '/diseno_experimental/variedad_parcela_var'

  # Variable con la identificación de cada roi
  id_roi_var = evaluacion_grp.createVariable('id_roi', 'i4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=-9999)
  id_roi_var.long_name = 'identificacion individual del roi'
  id_roi_var.comment = 'Identificación individual de cada roi en el ensayo'
  id_roi_var.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  id_roi_var.ancillary_variables = 'id_evaluacion id_asignacion /diseno_experimental/variedad_parcela_var'

  # --- SUBGRUPO ESPACIAL EVALUACIONES

  espacial_eva_e = evaluacion_grp.createGroup('espacial')
  espacial_eva_e.crs_spatial_references = 'crs_utm18n'
  espacial_eva_e.title = 'Componente espacial dinámico asociado con las evaluaciones'
  espacial_eva_e.description = 'Rasters y vectores generados en la evaluación como ortomosaicos multiespectrales, RGB o modelos de superficie'

  # Ortomosaico señal del sensor en cada dia de evaluación de cada ciclo
  om_ms_si_ae_ev = espacial_eva_e.createVariable('ortom_ms_evaluacion_signal', 'f4', ('y_r', 'x_r', 'num_ms_bandas', 'num_ciclos', 'num_dias_postcorte'), fill_value=-9999.0, zlib=True, chunksizes=(2048, 2048, 1, 1, 1))
  om_ms_si_ae_ev.long_name = 'Ortomosaico MS en señal de el sensor por ciclo y dia post corte'
  om_ms_si_ae_ev.comment = 'Ortomosaico MS sin calibración radiométrica del área experimental correspondiente a cada dia post corte en cada ciclo de evaluación'
  om_ms_si_ae_ev.units = '1'
  om_ms_si_ae_ev.crs_wkt = wkt_string
  om_ms_si_ae_ev.GeoTransform = geotransform_string
  om_ms_si_ae_ev.coordinates = 'y_r x_r num_ms_bandas num_ciclos num_dias_postcorte'
  om_ms_si_ae_ev.grid_mapping = 'crs_utm18n'

  # Ortomosaico multiespectral en cada dia de evaluación de cada ciclo
  om_ms_ref_ae_ev = espacial_eva_e.createVariable('ortom_ms_evaluacion_reflectividad', 'f4', ('y_r', 'x_r', 'num_ms_bandas','num_ciclos', 'num_dias_postcorte'), fill_value=-9999.0, zlib=True, chunksizes=(2048, 2048, 1, 1, 1))
  om_ms_ref_ae_ev.long_name = 'Ortomosaico MS de reflectividad en el sensor por ciclo y dia postcorte'
  om_ms_ref_ae_ev.comment = 'Ortomosaico MS con calibración radiométrica del área experimental correspondiente a cada dia post corte en cada ciclo de evaluación'
  om_ms_ref_ae_ev.units = '1'
  om_ms_ref_ae_ev.crs_wkt = wkt_string
  om_ms_ref_ae_ev.GeoTransform = geotransform_string
  om_ms_ref_ae_ev.coordinates = 'y_r x_r num_ms_bandas num_ciclos num_dias_postcorte'
  om_ms_ref_ae_ev.grid_mapping = 'crs_utm18n'

  # Ortomosaico RGB en cada día de evaluación de cada ciclo
  om_rgb_ae_ev = espacial_eva_e.createVariable('ortom_rgb_evaluacion', 'u1', ('y_r', 'x_r', 'num_rgb_bandas', 'num_dias_postcorte','num_ciclos'), fill_value=0, zlib=True, chunksizes=(2048, 2048, 1, 1, 1))
  om_rgb_ae_ev.long_name = 'Ortomosaico RGB poc ciclo y dia postcorte'
  om_rgb_ae_ev.comment = 'Ortomosaico RGB del área experimental asociado a la evaluacion correspondiente a cada dia post corte en cada ciclo de evaluación'
  om_rgb_ae_ev.crs_wkt = wkt_string
  om_rgb_ae_ev.GeoTransform = geotransform_string
  om_rgb_ae_ev.coordinates = 'y_r x_r num_rgb_bandas num_ciclos num_dias_postcorte'
  om_rgb_ae_ev.grid_mapping = 'crs_utm18n'

  # Raster con el modelo de superficie en cada día de evaluación de cada ciclo
  om_chm_ae_ev = espacial_eva_e.createVariable(
      'modelo_superficie_evaluacion', 'f4', ('y_r', 'x_r','num_ciclos', 'num_dias_postcorte'), fill_value=-9999.0, zlib=True, chunksizes=(2048, 2048, 1, 1))
  om_chm_ae_ev.long_name = 'Modelo de superficie por ciclo y dia postcorte'
  om_chm_ae_ev.comment = 'Modelo de superficie del área experimental asociado a la evaluacion correspondiente a cada dia post corte en cada ciclo de evaluación'
  om_chm_ae_ev.units = 'm'
  om_rgb_ae_ev.crs_wkt = wkt_string
  om_rgb_ae_ev.GeoTransform = geotransform_string
  om_chm_ae_ev.coordinates = 'y_r x_r num_ciclos num_dias_postcorte'
  om_chm_ae_ev.grid_mapping = 'crs_utm18n'

  # ROIS definidos por variables de indexado en la grilla principal

  # Máscara de los rásters de evaluación que define los pixeles con y sin datos en ROI
  roi_mask_var = espacial_eva_e.createVariable('mascara_rois', 'u1', ('y_r', 'x_r', 'num_ciclos', 'num_dias_postcorte'), fill_value=255, zlib=True, chunksizes=(2048, 2048, 1, 1))
  roi_mask_var.title = 'Mascaras de los ROI de evaluación'
  roi_mask_var.comment = 'Máscara de los rásters de evaluación que define los pixeles con y sin datos en ROI en toda el área experimental por ciclo y dia post corte'
  roi_mask_var.coordinates = 'y_r x_r num_ciclos num_dias_postcorte '
  roi_mask_var.grid_mapping = 'crs_utm18n'

  # Variable para alojar las coordenadas de los ROI
  roi_vect_coor = espacial_eva_e.createVariable('roi_coordenadas_vertices', 'f8', (   'num_ciclos','num_dias_postcorte', 'num_bloque', 'num_parcela', 'num_coord_xy', 'num_puntos'), fill_value=np.nan)
  roi_vect_coor.long_name = 'Coordenadas X Y de las de los roi utilizados para el muestreo en cada ciclo dia pos corte bloque y parcela'
  roi_vect_coor.units = 'm'
  roi_vect_coor.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela num_coord_xy num_puntos'
  roi_vect_coor.grid_mapping = 'crs_utm18n'
  roi_vect_coor.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'
  roi_vect_coor.featureType = 'polygon'

  # Variable para alojar los indices que identifican cada roi en la referencia de los rasters
  roi_rast_index = espacial_eva_e.createVariable('roi_indice_pixeles', 'i4', ('num_ciclos','num_dias_postcorte','num_bloque', 'num_parcela',   'num_slices'), fill_value=-9999)
  roi_rast_index.long_name = 'Indice para cortar las dimensiones globales a la extensión del área de los roi evaluados'
  roi_rast_index.comment = 'El índice es inclusivo para el inicio y exclusivo para el final'
  roi_rast_index.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela num_slices'
  roi_rast_index.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # SUBGRUPO METADATOS VUELOS

  # Creación de un nuevo subgrupo para los metadatos del vuelo
  metadatos_vuelo_grp = espacial_eva_e.createGroup('metadatos_vuelo')
  metadatos_vuelo_grp.title = "Metadatos de los vuelos UAV y procesamiento"
  metadatos_vuelo_grp.description = "Información detallada sobre cada misión de vuelo UAV y el procesamiento de los datos espaciales."
  metadatos_vuelo_grp.configuración_vuelos = 'Utilizando sistema de posicionamiento RTK y puntos de control fijos representados por 7 mojones distribuidos por el área experimental y georreferenciados por posicionamiento estático utilizando efemérides precisas en post procesamiento con tiempos de rastreo de 30 minutos respecto a la referencia del lote con lineas base de máximo 60 metros. Los vuelos se realizaron con seguimiento del terreno a partir de modelos digitales de terreno generados previamente con vuelos en el sentido contrario a la orientación pendiente para seguir lineas de altitud aproximadamente similar'

  # Variables para metadatos del vuelo dimensionadas por evento de evaluación
  flight_dims = ('num_ciclos', 'num_dias_postcorte')

  var_fecha_vuelo = metadatos_vuelo_grp.createVariable('fecha_hora_vuelo_inicio', str, ('num_ciclos', 'num_dias_postcorte'))
  var_fecha_vuelo.long_name = "Fecha y hora de inicio del vuelo UAV"
  var_fecha_vuelo.comment = "Formato ISO 8601: YYYY-MM-DDTHH:MM"
  var_fecha_vuelo.coordinates = 'num_ciclos num_dias_postcorte'

  var_plataforma = metadatos_vuelo_grp.createVariable('plataforma_uav', str, flight_dims)
  var_plataforma.long_name = "Marca y modelo de la plataforma UAV utilizada"

  var_sensor = metadatos_vuelo_grp.createVariable('sensor_uav', str, flight_dims)
  var_sensor.long_name = "Marca, modelo y número de serie del sensor utilizado"

  var_altitud = metadatos_vuelo_grp.createVariable('altitud_vuelo_mts_agl', 'f4', flight_dims, fill_value=np.nan)
  var_altitud.long_name = "Altitud de vuelo sobre el nivel del suelo (AGL)"
  var_altitud.units = "m"

  var_gsd = metadatos_vuelo_grp.createVariable('gsd_cm_px', 'f4', flight_dims, fill_value=np.nan)
  var_gsd.long_name = "Resolución espacial en el suelo (Ground Sampling Distance)"
  var_gsd.units = "cm/pixel"

  var_cielo = metadatos_vuelo_grp.createVariable('condiciones_cielo', str, flight_dims)
  var_cielo.long_name = "Condiciones del cielo durante el vuelo"
  var_cielo.comment = "Ej: despejado, parcialmente nublado, nublado, % de cobertura nubosa si se conoce"

  var_software = metadatos_vuelo_grp.createVariable('software_procesamiento_fotogrametria', str, flight_dims)
  var_software.long_name = "Software y versión utilizados para el procesamiento fotogramétrico"

  var_cal_rad = metadatos_vuelo_grp.createVariable('metodo_calibracion_radiometrica', str, flight_dims)
  var_cal_rad.long_name = "Método utilizado para la calibración radiométrica"

  var_notas_vuelo = metadatos_vuelo_grp.createVariable('notas_vuelo_procesamiento', str, flight_dims)
  var_notas_vuelo.long_name = "Notas adicionales relevantes sobre el vuelo o el procesamiento"

  # ---// SUBGRUPO PRODUCTOS DERIVADOS

  raster_derivados = evaluacion_grp.createGroup('raster_derivados')
  raster_derivados.title = 'Rasters derivados de vuelos UAV'
  raster_derivados.long_name = 'Rasters generados por procesamiento de productos primarios UAV '
  raster_derivados.comment = 'Rasters que se derivan del procesamiento de los vuelos UAV y productos primarios generados tales como ínidices de vegetación, modelos de altura de dosel, e información textural'
  raster_derivados.crs_spatial_references = 'crs_utm18n'

  # Variable de indices espectrales estimados
  spectral_indices = raster_derivados.createVariable('indices_espectrales_evaluacion', 'f4', ('y_r', 'x_r', 'num_spectral_indices', 'num_ciclos', 'num_dias_postcorte'), chunksizes=(2048, 2048, 1, 1, 1), zlib=True, fill_value=-9999.0)
  spectral_indices.long_name = 'Indices espectrales estimados'
  spectral_indices.units = '1'
  spectral_indices.coordinates = 'y_r x_r num_spectral_indices num_ciclos num_dias_postcorte'
  spectral_indices.grid_mapping = 'crs_utm18n'
  spectral_indices.comment = 'Indices espectrales estimados para el área experimental según la descripción de la variable coordenada indices_espectrales_coord incluye los indices: ndvi, gndvi, ndre, ci_rededge, ci_green, rvi, sr_rededge, evi2, savi, msavi, grvi'
  spectral_indices.crs_wkt = wkt_string
  spectral_indices.GeoTransform = geotransform_string

  # Variable de modelos de altura de dosel
  dosel_model = raster_derivados.createVariable('modelo_altura_dosel_evaluacion', 'f4', ('y_r', 'x_r', 'num_ciclos', 'num_dias_postcorte'), chunksizes=(2048, 2048, 1, 1), zlib=True, fill_value=-9999.0)
  dosel_model.long_name = 'Modelo de altura de dosel por ciclo y dia post corte'
  dosel_model.units = 'm'
  dosel_model.coordinates = 'y_r x_r num_ciclos num_dias_postcorte'
  dosel_model.grid_mapping = 'crs_utm18n'
  dosel_model.comment = 'Modelo de altura de dosel estimado como la diferencia entre el modelo de superficie y modelo digital de terreno por ciclo y dias post corte. Estimado para toda el área experimental'
  dosel_model.crs_wkt = wkt_string
  dosel_model.GeoTransform = geotransform_string

  # Variables GLCM Gray Level Co-ocurrence matrix a nivel de parcela.
  glcm_features_parcela = raster_derivados.createVariable('glcm_valores_parcela_evaluacion', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela','num_glcm_estadisticos',), fill_value=np.nan)
  glcm_features_parcela.long_name = 'GLCM features por parcela'
  glcm_features_parcela.units = '1'
  glcm_features_parcela.comment = 'GLCM features por parcela según la descripción de la variable coordenada '
  glcm_features_parcela.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela num_glcm_estadisticos'

  # Variables GLCM Gray Level Co-ocurrence matrix a nivel de roi.
  glcm_features_roi = raster_derivados.createVariable('glcm_valores_roi_evaluacion', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela','num_glcm_estadisticos',), fill_value=np.nan)
  glcm_features_roi.long_name = 'GLCM features por roi en cada parcela'
  glcm_features_roi.units = '1'
  glcm_features_roi.comment = 'GLCM features por roi en cada parcela según la descripción de la variable coordenada '
  glcm_features_roi.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela num_glcm_estadisticos'

  # --- // SUBGRUPO CLIMA ---

  clima_grp = evaluacion_grp.createGroup('clima')
  clima_grp.long_name = 'Datos meteorológicos en los ciclos de evaluación'
  clima_grp.comment = 'Datos meteorológicos obtenidos durante los ciclos de evaluación.'

  # Función para crear variables de series de tiempo del clima indexadas por ciclo y la dimension temporal
  def create_climate_timeseries_cycled(group, base_name, long_name_val, units_val, interval_val, time_dim_name, standard_name_val="", cell_method_val=None):

    """Crea series de tiempo para las variables climáticas dentro de un grupo NetCDF, indexadas por ciclo y hora.

    Args:
        group (netCDF4.Group): El grupo NetCDF donde las variables serán creadas.
        base_name (str): El nombre base para el timestamp y las variables cpm valores. (e.g., 'temp_horas').
        long_name_val (str): Un nombre largo descriptivo para las variables de valor.
        units_val (str): Las unidades de la variable de valor.
        interval_val (str): Descripción del intervalo de medida (e.g., 'Daily total').
        time_dim_name (str): El nombre de la dimension temporal a usar (e.g., 'num_tiempo_hora', 'num_tiempo_diario').
        standard_name_val (str, optional): El nombre estandar CF para la variable valor. Por defecto "".
        cell_method_val (str, optional): El valor para el atributo cell_methods  (e.g., 'time: maximum'). Por defecto None.

    Returns:
        tuple: Una tupla que contiene la variable timestamp  (netCDF4.Variable) y los valores de la variable (netCDF4.Variable).

    La función crea dos variables NetCDF dentro del grupo especificado: una para timestamps (como strings) y una para los correspondientes valores climáticos
    Ambas variables están indexadas por la dimension 'num_ciclos' y una dimension de tiempo especificada. Las convensiones CF son aplicadas, incluyendo, long_name, units, standard_name, measurement_interval, and a link  between the values and timestamp variables using the ancillary_variables attribute.
    """
    # Usar la dimension num_ciclos y la dimension de tiempo especificada
    dims = ('num_ciclos', time_dim_name,)

    # Utilizar string para momentos de tiempo. Crear una variable timestamp indexada por las dimensiones ciclo y tiempo.
    ts_var = group.createVariable(f'marca_tiempo_{base_name}', str, dims)
    ts_var.long_name = f"Marca de tiempo para {long_name_val}"
    ts_var.comment = "Strings de marca de tiempo en formato 'YYYY-MM-DD HH:MM:SS' para cada ciclo y paso de tiempo."

    # Crear la variable de valores indexada por las dimensiones ciclo y tiempo
    val_var = group.createVariable(f'valores_{base_name}', 'f4', dims, zlib=True, fill_value=np.nan)
    val_var.long_name = long_name_val
    val_var.units = units_val

    # Asociar con la variable estampa de tiempo
    val_var.ancillary_variables = f'marca_tiempo_{base_name}'
    if standard_name_val:
        val_var.standard_name = standard_name_val
    val_var.measurement_interval = interval_val

    # Asociar con las coordenadas del ciclo
    val_var.coordinates = 'num_ciclos ' + time_dim_name
    if cell_method_val:
        val_var.cell_methods = cell_method_val

    return ts_var, val_var

  #  Crear las variables climáticas utilizando la función

  # Temperatura del aire en horas específicas
  times_ts, ts_hourly = create_climate_timeseries_cycled(clima_grp, 'temperatura_seca', 'Air Temperature at Specific Hours', 'degree_Celsius', 'Measurements at 07:00, 13:00, 18:00 local time', 'num_tiempo_hora', 'air_temperature')
  ts_hourly.coordinates = 'num_ciclos num_tiempo_hora'

  # Temperatura máxima del aire diaria
  times_ts_max, ts_max_daily = create_climate_timeseries_cycled(clima_grp, 'ts_max_dia', 'Maximum Daily Air Temperature', 'degree_Celsius', 'Daily maximum', 'num_tiempo_diario', 'air_temperature_maximum', cell_method_val='num_tiempo_diario: maximum')
  ts_max_daily.coordinates = 'num_ciclos num_tiempo_diario'

  # Temperatura mínima del aire diaria
  times_ts_min, ts_min_daily = create_climate_timeseries_cycled(clima_grp, 'ts_min_dia', 'Minimum Daily Air Temperature', 'degree_Celsius', 'Daily minimum', 'num_tiempo_diario', 'air_temperature_minimum', cell_method_val='num_tiempo_diario: minimum')
  ts_min_daily.coordinates = 'num_ciclos num_tiempo_diario'

  # Humedad relativa horaria
  times_hr, hr_hourly = create_climate_timeseries_cycled(clima_grp, 'hr', 'Relative Humidity', 'percent', 'Hourly average or instantaneous', 'num_tiempo_hora', 'relative_humidity')
  hr_hourly.coordinates = 'num_ciclos num_tiempo_hora'

  # Precipitación acumulada diaria
  times_rain, rain_daily = create_climate_timeseries_cycled(clima_grp, 'prec_dia', 'Precipitation Amount', 'mm', 'Daily total', 'num_tiempo_diario', 'precipitation_amount',  cell_method_val='num_tiempo_diario: sum')
  rain_daily.coordinates = 'num_ciclos num_tiempo_diario'

  # --- // SUBRUPO MEDICIONES CAMPO ---

  gt_cam_grp = evaluacion_grp.createGroup('mediciones_campo')
  gt_cam_grp.title = 'Mediciones de campo de la verdad terrestre'
  gt_cam_grp.comment = 'Mediciones de verdad terrestre en campo, indexadas por ciclo y día de evaluación.'
  gt_cam_grp.crs_spatial_references = dis_exp_grp.crs_spatial_references

  # Variables relacionadas con la medición en campo
  fecha_evaluaciones_campo = gt_cam_grp.createVariable('fecha_evaluacion_campo', str, ('num_ciclos', 'num_dias_postcorte'))
  fecha_evaluaciones_campo.long_name = 'Fecha de las evaluaciones de campo'
  fecha_evaluaciones_campo.comment = 'Fecha de las evaluaciones realizado en las parcelas al inicio de cada ciclo de evaluación. Strings de marca de tiempo en formato YYYY-MM-DD HH:MM:SS para cada ciclo y paso de tiempo'
  fecha_evaluaciones_campo.coordinates = 'num_ciclos num_dias_postcorte'

  notas_evaluaciones_campo = gt_cam_grp.createVariable('notas_evaluacion_campo', str, ('num_ciclos', 'num_dias_postcorte'))
  notas_evaluaciones_campo.long_name = 'Notas de las evaluaciones de campo'
  notas_evaluaciones_campo.comment = 'Texto libre para observaciones relevantes o incidentes durante las mediciones de campo en cada fecha de evaluación'
  notas_evaluaciones_campo.coordinates = 'num_ciclos num_dias_postcorte'

  # Variables de la verdad en tierra indicadas por el número de ciclos días pos corte numero de bloques y numero de parcelas por bloque

  # Variables coordenadas por bloque y parcela ID (Definidos globalmente en diseno_experimental/planteamiento_experimento)

  # Vigor
  gt_vigor_var = gt_cam_grp.createVariable('vigor', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_vigor_var.long_name = gt_cam_atributos['descripcion']['vigor']
  gt_vigor_var.units = gt_cam_atributos['unidades']['vigor']
  gt_vigor_var.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_vigor_var.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # Cobertura
  gt_cobertura_var = gt_cam_grp.createVariable('cobertura', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_cobertura_var.long_name = gt_cam_atributos['descripcion']['cobertura']
  gt_cobertura_var.units = gt_cam_atributos['unidades']['cobertura']
  gt_cobertura_var.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_cobertura_var.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # Severidad de pestes
  gt_plagas_severidad_var = gt_cam_grp.createVariable('plagas_severidad', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_plagas_severidad_var.long_name = gt_cam_atributos['descripcion']['plagas_severidad']
  gt_plagas_severidad_var.units = gt_cam_atributos['unidades']['plagas_severidad']
  gt_plagas_severidad_var.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_plagas_severidad_var.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # Severidad de enfermedades
  gt_enfermedad_severidad_var = gt_cam_grp.createVariable('enfermedad_severidad', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_enfermedad_severidad_var.long_name = gt_cam_atributos['descripcion']['enfermedad_severidad']
  gt_enfermedad_severidad_var.units = gt_cam_atributos['unidades']['enfermedad_severidad']
  gt_enfermedad_severidad_var.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_enfermedad_severidad_var.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # Porcentaje de floración
  gt_floracion_var = gt_cam_grp.createVariable('floracion', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_floracion_var.long_name = gt_cam_atributos['descripcion']['floracion']
  gt_floracion_var.units = gt_cam_atributos['unidades']['floracion']
  gt_floracion_var.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_floracion_var.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # Total peso verde biomasa
  gt_peso_verde_total = gt_cam_grp.createVariable('total_peso_verde', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_peso_verde_total.long_name = gt_cam_atributos['descripcion']['total_peso_verde']
  gt_peso_verde_total.units = gt_cam_atributos['unidades']['total_peso_verde']
  gt_peso_verde_total.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_peso_verde_total.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # Peso de muestra humeda biomasa
  gt_peso_verde_muestra = gt_cam_grp.createVariable('muestra_peso_verde', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_peso_verde_muestra.long_name = gt_cam_atributos['descripcion']['muestra_peso_verde']
  gt_peso_verde_muestra.units = gt_cam_atributos['unidades']['muestra_peso_verde']
  gt_peso_verde_muestra.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_peso_verde_muestra.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # Peso de muestra seca biomasa
  gt_peso_seco_muestra = gt_cam_grp.createVariable('muestra_peso_seco', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_peso_seco_muestra.long_name = gt_cam_atributos['descripcion']['muestra_peso_seco']
  gt_peso_seco_muestra.units = gt_cam_atributos['unidades']['muestra_peso_seco']
  gt_peso_seco_muestra.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_peso_seco_muestra.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # Descuento de saco
  gt_descuento_saco = gt_cam_grp.createVariable('descuento_saco', 'u1', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=255)
  gt_descuento_saco.long_name = gt_cam_atributos['descripcion']['descuento_saco']
  gt_descuento_saco.units = gt_cam_atributos['unidades']['descuento_saco']
  gt_descuento_saco.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_descuento_saco.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # Peso de saco
  gt_peso_saco = gt_cam_grp.createVariable('peso_saco_total', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_peso_saco.long_name = gt_cam_atributos['descripcion']['peso_saco']
  gt_peso_saco.units = gt_cam_atributos['unidades']['peso_saco']
  gt_peso_saco.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_peso_saco.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # Peso bolsa muestra
  gt_peso_bolsa_muestra = gt_cam_grp.createVariable('peso_bolsa', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_peso_bolsa_muestra.long_name = gt_cam_atributos['descripcion']['peso_bolsa']
  gt_peso_bolsa_muestra.units = gt_cam_atributos['unidades']['peso_bolsa']
  gt_peso_bolsa_muestra.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_peso_bolsa_muestra.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # Altura
  gt_altura_muestra = gt_cam_grp.createVariable('altura', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela', 'num_medidas_repetidas'), fill_value=np.nan)
  gt_altura_muestra.long_name = gt_cam_atributos['descripcion']['altura']
  gt_altura_muestra.units = gt_cam_atributos['unidades']['altura']
  gt_altura_muestra.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela num_medidas_repetidas'
  gt_altura_muestra.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # SPAD
  gt_spad_muestra = gt_cam_grp.createVariable('spad', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela', 'num_medidas_repetidas'), fill_value=np.nan)
  gt_spad_muestra.long_name = gt_cam_atributos['descripcion']['valor_spad']
  gt_spad_muestra.units = gt_cam_atributos['unidades']['valor_spad']
  # Adicionar el atributo coordenadas de 'gt_spad_muestra'
  gt_spad_muestra.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela num_medidas_repetidas'
  gt_spad_muestra.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # Peso de suelo sólo medido a nivel de bloque
  # Peso de muestra humeda suelo
  gt_peso_humedo_suelo = gt_cam_grp.createVariable('peso_suelo_fresco', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque'), fill_value=np.nan)
  gt_peso_humedo_suelo.long_name = 'Peso húmedo de la muestra de suelo'
  gt_peso_humedo_suelo.units = 'kg'
  gt_peso_humedo_suelo.comment = 'Peso húmedo de la muestra de suelo descontando empaques obtenido de submuestreo a nivel de bloque en cada ciclo y dia de evaluación'
  gt_peso_humedo_suelo.coordinates = 'num_ciclos num_dias_postcorte num_bloque'
  gt_peso_humedo_suelo.ancillary_variables = '/evaluaciones/id_evaluacion'

  # Peso de muestra seca suelo
  gt_peso_seco_suelo = gt_cam_grp.createVariable('peso_suelo_seco', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque'), fill_value=np.nan)
  gt_peso_seco_suelo.long_name = 'Peso seco de la muestra de suelo'
  gt_peso_seco_suelo.units = 'kg'
  gt_peso_seco_suelo.coordinates = 'num_ciclos num_dias_postcorte num_bloque'
  gt_peso_seco_suelo.ancillary_variables = '/evaluaciones/id_evaluacion'

  # --- // SUBGRUPO RESULTADOS DE LABORATORIO

  gt_lab_grp = evaluacion_grp.createGroup('bromatologia')
  gt_lab_grp.title = 'Calidad nutricional de las muestras evaluadas en laboratorio'
  gt_lab_grp.comment = 'Resultados de los análisis de laboratorio en ciclos seleccionados indexados por ciclo y día de evaluación.'

  # Proteina cruda
  gt_pc = gt_lab_grp.createVariable('proteina_cruda', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_pc.long_name = gt_lab_atributos['descripcion']['proteina_cruda']
  gt_pc.units = gt_lab_atributos['unidades']['proteina_cruda']
  gt_pc.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_pc.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # FDN
  gt_fdn = gt_lab_grp.createVariable('fdn', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_fdn.long_name = gt_lab_atributos['descripcion']['fdn']
  gt_fdn.units = gt_lab_atributos['unidades']['fdn']
  gt_fdn.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_fdn.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # FDA
  gt_fda = gt_lab_grp.createVariable('fda', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_fda.long_name = gt_lab_atributos['descripcion']['fda']
  gt_fda.units = gt_lab_atributos['unidades']['fda']
  gt_fda.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_fda.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

  # DIVMS
  gt_dims = gt_lab_grp.createVariable('divms', 'f4', ('num_ciclos', 'num_dias_postcorte', 'num_bloque', 'num_parcela'), fill_value=np.nan)
  gt_dims.long_name = gt_lab_atributos['descripcion']['digestibilidad_ms']
  gt_dims.units = gt_lab_atributos['unidades']['digestibilidad_ms']
  gt_dims.coordinates = 'num_ciclos num_dias_postcorte num_bloque num_parcela'
  gt_dims.ancillary_variables = '/evaluaciones/id_evaluacion /evaluaciones/id_asignacion /evaluaciones/id_roi /diseno_experimental/variedad_parcela_var'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Shape of x_coords_centers: (9098,)
Sample of x_coords_centers: [517476.01561331 517476.02561331 517476.03561331 517476.04561331
 517476.05561331]
Shape of y_coords_centers: (8159,)
Sample of y_coords_centers: [716481.13650014 716481.12650014 716481.11650014 716481.10650014
 716481.09650014]
Attempting to write to x_r_coord_var and y_r_coord_var...
Successfully wrote to x_r and y_r coordinate variables.
